In [1]:
import pymongo
import requests
from bs4 import BeautifulSoup as bs
import GetOldTweets3 as got
import time

from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tag import pos_tag
import re, string
from nltk.tokenize import word_tokenize

In [2]:
# used to normalize raw tweets
def clean_data(tweet_tokens, stop_words = ()):

    cleaned_tokens = []

    for token, tag in pos_tag(tweet_tokens):
        # removing unwanted symbols and patterns from tokens using regular expressions
        token = re.sub("http[s]?://+(?:[a-zA-Z]|[0-9]|[$-_@.&+#]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+","", token)
        token = re.sub(".(.co/o).","", token)
        token = re.sub("(@[A-Za-z0-9_]+)","", token)
        token = re.sub("\w*\d\w*","",token)
        token = re.sub('[‘’“”…#–/]', '', token)
#         token = re.sub("http[s]?","",token)
        
        # assigning new pos tags for WordNetLemmatizer() function
        if tag.startswith("NN"):
            pos = "n"
        elif tag.startswith("VB"):
            pos = "v"
        else:
            pos = "a"
            
        # lemmatizing tokens (running=run)
        lemmatizer = WordNetLemmatizer()
        token = lemmatizer.lemmatize(token, pos)
        
        # dropping puncuation and stop words
        if len(token) > 0 and token not in string.punctuation and token.lower() not in stop_words:
            cleaned_tokens.append(token.lower())
    return cleaned_tokens

### Connecting to MongoDB

In [3]:
conn = "mongodb://localhost:27017"
client = pymongo.MongoClient(conn)
db = client.twitter_db

### Scraping Tweets

In [4]:
import datetime
datetime.datetime.utcnow().year

2020

In [5]:
def get_tweets(search_term):
    month = datetime.datetime.utcnow().month
    year = datetime.datetime.utcnow().year
    
    all_tweets = []
    count = 0
    # setting parameters
    tweetCriteria = got.manager.TweetCriteria() \
        .setQuerySearch(search_term)\
        .setSince(f"{year}-{month}-01")\
        .setUntil(f"{year}-{month}-28")\
        .setMaxTweets(500)

    # scraping tweets
    tweets = got.manager.TweetManager.getTweets(tweetCriteria)

    # adding all tweets to list
    all_tweets.append(tweets)

    return all_tweets

In [7]:
search_term = "cheesecake factory"
month = datetime.datetime.utcnow().month
year = datetime.datetime.utcnow().year

all_tweets = []
count = 0
# setting parameters
tweetCriteria = got.manager.TweetCriteria() \
    .setQuerySearch(search_term) \
    .setSince(f"{year}-{month}-01") \
    .setUntil(f"{year}-{month}-28") \
    .setWithin("100mi") \
    .setMaxTweets(20)
#     .setNear(city) \
#                 .setTopTweets(True) \

# scraping tweets
tweets = got.manager.TweetManager.getTweets(tweetCriteria)

# adding all tweets to list
all_tweets.append(tweets)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



An error occured during an HTTP request: HTTP Error 404: Not Found
Try to open in browser: https://twitter.com/search?q=cheesecake%20factory%20since%3A2020-9-01%20until%3A2020-9-28&src=typd
Traceback (most recent call last):
  File "/Users/loganbon/opt/anaconda3/lib/python3.7/site-packages/GetOldTweets3/manager/TweetManager.py", line 343, in getJsonResponse
    response = opener.open(url)
  File "/Users/loganbon/opt/anaconda3/lib/python3.7/urllib/request.py", line 531, in open
    response = meth(req, response)
  File "/Users/loganbon/opt/anaconda3/lib/python3.7/urllib/request.py", line 641, in http_response
    'http', request, response, code, msg, hdrs)
  File "/Users/loganbon/opt/anaconda3/lib/python3.7/urllib/request.py", line 569, in error
    return self._call_chain(*args)
  File "/Users/loganbon/opt/anaconda3/lib/python3.7/urllib/request.py", line 503, in _call_chain
    result = func(*args)
  File "/Users/loganbon/opt/anaconda3/lib/python3.7/urllib/request.py", line 649, in htt

SystemExit: 

/Users/loganbon/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3339: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [15]:
restaurants = get_tweets("cheesecake factory")

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



An error occured during an HTTP request: HTTP Error 404: Not Found
Try to open in browser: https://twitter.com/search?q=cheesecake%20factory%20since%3A2020-9-01%20until%3A2020-9-28&src=typd
Traceback (most recent call last):
  File "/Users/loganbon/opt/anaconda3/lib/python3.7/site-packages/GetOldTweets3/manager/TweetManager.py", line 343, in getJsonResponse
    response = opener.open(url)
  File "/Users/loganbon/opt/anaconda3/lib/python3.7/urllib/request.py", line 531, in open
    response = meth(req, response)
  File "/Users/loganbon/opt/anaconda3/lib/python3.7/urllib/request.py", line 641, in http_response
    'http', request, response, code, msg, hdrs)
  File "/Users/loganbon/opt/anaconda3/lib/python3.7/urllib/request.py", line 569, in error
    return self._call_chain(*args)
  File "/Users/loganbon/opt/anaconda3/lib/python3.7/urllib/request.py", line 503, in _call_chain
    result = func(*args)
  File "/Users/loganbon/opt/anaconda3/lib/python3.7/urllib/request.py", line 649, in htt

SystemExit: 

### Adding to MongoDB

In [91]:
for tweet_bundle in restaurant:
    for tweet in tweet_bundle:
        # extracting information
        date = tweet.date
        tweet_text = tweet.text
        num_favorites = tweet.favorites
        num_replies = tweet.replies
        num_retweets = tweet.retweets
        author_username = tweet.username
        tweet_id = tweet.id

        # cleaning tweet text 
        clean_tokenized_tweet = clean_data(word_tokenize(tweet_text))

        # creating dictionary object
        post = {
            "search_term": search_term,
            "date_posted": date,
            "original_tweet": tweet_text,
            "clean_tokenized_tweet": clean_tokenized_tweet,
            "num_of_favorites": num_favorites,
            "num_of_replies": num_replies,
            "num_retweets": num_retweets,
            "username": author_username,
            "tweet_id": tweet_id
        }

        # inserting object into database
        db.restaurants.insert_one(post)

In [11]:
def get_tweets(username, top_only, start_date, end_date, max_tweets):
   
    # specifying tweet search criteria 
    tweetCriteria = got.manager.TweetCriteria().setUsername(username)\
                          .setTopTweets(top_only)\
                          .setSince(start_date)\
                          .setUntil(end_date)\
                          .setMaxTweets(max_tweets)
    
    # scraping tweets based on criteria
    tweet = got.manager.TweetManager.getTweets(tweetCriteria)
    
    # creating list of tweets with the tweet attributes 
    # specified in the list comprehension
    text_tweets = [[tw.username,
                tw.text,
                tw.date,
                tw.retweets,
                tw.favorites,
                tw.mentions,
                tw.hashtags] for tw in tweet]
    
    # creating dataframe, assigning column names to list of
    # tweets corresponding to tweet attributes
    news_df = pd.DataFrame(text_tweets, 
                            columns = ['User', 'Text','Date',      'Favorites', 'Retweets', 'Mentions', 'HashTags'])
    
    return news_df